In [1]:
import numpy as np
import pandas as pd
from scipy.stats import uniform
import cbadc as cb
from itertools import product
from numpy.linalg import LinAlgError
from scipy.stats import qmc

In [2]:
def generate_dataset(form, order, implementation, num_samples, min_snr, parameter_ranges, discrete_params):
    """
    Generate a dataset based on the selected form, order, implementation
    
    Args:
    - form (str): The (continuous) form of the filter ('FB', 'FF')
    - order (int): The order of the filter (2, 3, 4)
    - implementation (str): The implementation type ('Active-RC', 'Gm-C')
    - num_samples (int): Initial number of samples to run LHS
    - min_snr (float): Minimum SNR accepted.
    - parameter_ranges (list): List with the range of each parameter
    - discrete_parameters (list): List with the discrete parameters and each values
    
    Returns:
    - dataset (pd.DataFrame): A DataFrame containing the generated dataset,
                              named: dataset_form_order_implementation.csv
    """
    
    cont = 0 # counter to reach at least num_samples samples in the final dataset
    
    # initialize lists to store the parameters and corresponding SNR values after simulation:
    snr_values = []
    all_params = []
    
    while cont <= num_samples:
        # Latin Hypercube Sampling (LHS) to sample the parameter space:
        lhs_samples = lhs_sampling(parameter_ranges, num_samples, discrete_params)

        # iterate through each set of parameters from the LHS samples:
        for sample in lhs_samples:
            # map the sampled values to parameter names:
            if implementation == 'Active-RC':
                params = {'Bw': sample[0], 'osr': int(sample[1]), 'nlev': int(sample[2]), 'Hinf': sample[3], 
                          'Cint': sample[4], 'gm': sample[5], 'Ro': sample[6], 'Co': sample[7]}
            elif implementation == 'Gm-C':
                params = {'Bw': sample[0], 'osr': int(sample[1]), 'nlev': int(sample[2]), 'Hinf': sample[3], 
                          'Cint': sample[4], 'Ro': sample[5], 'Cp': sample[6], 'v_n': sample[7], 
                          'slew_rate': sample[8], 'output_swing': sample[9]}

            # apply parameter relationships filters to ensure validity (not implemented yet):
            if not filter_parameter_relationships(params, implementation):
                continue # skip this sample if it doesn't pass the filter

            # simulate the circuit and get the SNR value:
            snr = simulate_circuit(form, order, implementation, params)

            # store the the parameters and corresponding SNR value:
            if snr is not None and np.isfinite(snr) and snr > min_snr:
                snr_values.append(snr)
                all_params.append(params)
                cont += 1
    
    # create a DataFrame and save it as CSV:
    dataset = pd.DataFrame(all_params)
    dataset['SNR'] = snr_values
    filename = f"datasets/dataset_{form}_{order}_{implementation}.csv"
    dataset.to_csv(filename, index=False)
    print(f"\n\tDataset saved as {filename}\n")
    
    return dataset

In [3]:
def lhs_sampling(parameter_ranges, num_samples, discrete_params):
    """
    Latin Hypercube Sampling (LHS) for a given set of parameter ranges,
    handling both continuous and discrete parameters

    Args:
    - parameter_ranges (list): List of tuples [(min, max), ...] defining the range for each parameter
    - num_samples (int): Number of samples to generate
    - discrete_params (list): List of tuples [(index, values)] where:
        - index: The position of the discrete parameter in parameter_ranges
        - values: A list of possible discrete values

    Returns:
    - samples (list): A NumPy array of shape (num_samples, len(parameter_ranges)) containing the generated samples
    """

    # separate continuous and discrete parameters:
    continuous_params = [p for i, p in enumerate(parameter_ranges) if i not in [d[0] for d in discrete_params]]
    discrete_indices = {d[0]: d[1] for d in discrete_params}

    # create an LHS design only for continuous parameters:
    sampler = qmc.LatinHypercube(d=len(continuous_params)) # d = number of continuous parameters
    sample_cont = sampler.random(num_samples) # generate LHS samples in [0,1] range
    
    # rescale LHS samples to match the specified ranges for continuous parameters:
    cont_scaled = qmc.scale(sample_cont, 
                            [p[0] for p in continuous_params], # min values
                            [p[1] for p in continuous_params]) # max values
    
    # initialize the result matrix:
    samples = np.zeros((num_samples, len(parameter_ranges)))

    # insert continuous values into the result matrix:
    cont_idx = 0 # index to track continuous parameters in cont_scaled
    for i, p in enumerate(parameter_ranges):
        if i in discrete_indices: # skip discrete parameters
            continue
        samples[:, i] = cont_scaled[:, cont_idx] # assign LHS-sampled values
        cont_idx += 1

    # insert discrete parameters using random selection with replacement:
    for idx, values in discrete_indices.items():
        samples[:, idx] = np.random.choice(values, size=num_samples, replace=True)

    return samples

In [4]:
# LHS test:

parameter_ranges = [
            (1e3, 1e9), # Bw (Bandwidth)
            [8, 16, 32, 64, 128, 256], # osr (Oversampling Ratio) - Discrete
            [2, 3, 4, 5], # nlev (Number of Levels) - Discrete
            (1e-3, 1e1), # Hinf (Infinity Gain)
            (1e-12, 1e-6), # Cint (Integration Capacitance)
            (1e-3, 1e-1), # gm (Transconductance)
            (1e3, 1e6), # Ro (Output Resistance)
            (1e-12, 1e-6), # Co (Output Capacitance)
        ]
discrete_params = [
        (1, [8, 16, 32, 64, 128, 256]), # osr (discrete)
        (2, [2, 3, 4, 5]) # nlev (discrete)
    ]

num_samples = 100 # number of samples
lhs_samples = lhs_sampling(parameter_ranges, num_samples, discrete_params)

print(lhs_samples)
print(len(lhs_samples))

[[2.51961566e+08 1.28000000e+02 3.00000000e+00 4.07627118e+00
  4.24527468e-07 6.68838935e-02 7.09258853e+05 5.33346469e-07]
 [6.44550286e+08 2.56000000e+02 5.00000000e+00 4.10692814e-01
  3.49044719e-07 4.33035572e-02 4.64040859e+05 7.80558826e-07]
 [4.76299260e+08 1.28000000e+02 5.00000000e+00 4.11441667e+00
  5.31389622e-07 7.31428479e-02 1.22986710e+05 6.73703035e-07]
 [1.28887170e+08 6.40000000e+01 4.00000000e+00 1.60951473e+00
  5.52931049e-07 7.72803022e-02 6.94705096e+05 1.39593999e-07]
 [2.88427013e+08 1.28000000e+02 3.00000000e+00 3.87582102e+00
  4.54443548e-07 4.89698537e-03 6.19535800e+05 3.98390790e-07]
 [6.87285949e+08 1.60000000e+01 3.00000000e+00 5.88903561e+00
  5.24087797e-07 9.86032928e-02 9.49601804e+05 6.03361338e-07]
 [4.87633963e+08 1.28000000e+02 4.00000000e+00 8.60891859e+00
  8.58100993e-07 4.91913376e-02 7.74987362e+05 3.24029955e-07]
 [3.88578958e+08 6.40000000e+01 3.00000000e+00 6.00955524e-03
  2.61417937e-07 6.32884985e-02 6.85410777e+05 7.06215447e-07]


In [5]:
def simulate_circuit(form, order, implementation, params):
    """
    Simulate the circuit and return the SNR value based on the selected
    form, order and implementation
    
    Args:
    - form (str): The (continuous) form of the filter ('FB', 'FF')
    - order (int): The order of the filter (2, 3, 4)
    - implementation (str): The implementation type ('Active-RC', 'Gm-C')
    - params (dict): A dictionary containing the parameters for the simulation
    
    Returns:
    - max_snr (float): The maximum SNR value obtained from the simulation
    """
    
    # define parameters:
    Bw = params.get('Bw')
    osr = params.get('osr')
    nlev = params.get('nlev')
    Hinf = params.get('Hinf')
    f0 = 0.
    tdac = [0, 1]
    
    # compute the sampling frequency:
    fs = Bw * osr * 2
    
    # synthesize the optimal NTF (Noise Transfer Function)
    ntf = cb.delsig.synthesizeNTF(order, osr, 2, Hinf, f0)
    
    # create analog frontend:
    dt = 1.0 / fs
    ABCDc, tdac2 = cb.delsig.realizeNTF_ct(ntf, form, tdac)
    analog_frontend = cb.AnalogFrontend.ctsdm(ABCDc, tdac2, dt, nlev)
    analog_frontend.dt = 1.0 / fs
    
    # run the SNR simulation (depending on the implementation):
    if implementation == 'Active-RC':
        max_snr = simulate_snr_active_rc(params, order, osr, analog_frontend)
    elif implementation == 'Gm-C':
        max_snr = simulate_snr_gm_c(params, order, osr, analog_frontend)    
    else:
        raise ValueError(f"Unknown implementation type: {implementation}")
    
    return max_snr



def simulate_snr_active_rc(params, order, osr, analog_frontend):
    """
    Simulate the SNR for Active-RC implementation
    
    Args:
    - params (dict): The parameter set
    - order (int): The order of the filter (2, 3, 4)
    - osr (int): The oversampling ratio
    
    Returns:
    - snr (float): The simulated SNR value.
    """
    
    # define parameters:
    Cint = np.ones(order) * params.get('Cint')
    gm = np.ones(order) * params.get('gm')
    Ro = np.ones(order) * params.get('Ro')
    Co = np.ones(order) * params.get('Co')
    
    # simulate SNR:
    try:
        ActiveRC_analog_frontend = cb.ActiveRC(
            analog_frontend, Cint, gm, Ro, Co
        )
                
        snr_ActiveRC, amp_ActiveRC, _ = ActiveRC_analog_frontend.simulateSNR(osr)
        max_snr_ActiveRC = max(snr_ActiveRC)
        
        if not np.isnan(max_snr_ActiveRC):
            print(f"Achieved SNR: {max_snr_ActiveRC} db")
            return max_snr_ActiveRC
        else:
            return None
        
    except (LinAlgError, ValueError) as e:
        print(f"Error with Active-RC parameters {params}: {e}")
        return None



# Function to simulate SNR for Gm-C
def simulate_snr_gm_c(params, order, osr, analog_frontend):
    """
    Simulate the SNR for Gm-C implementation
    
    Args:
    - params (dict): The parameter set
    - order (int): The order of the filter (2, 3, 4)
    - osr (int): The oversampling ratio
    
    Returns:
    - snr (float): The simulated SNR value
    """ 
    
    # define parameters:
    Cint = np.ones(order) * params.get('Cint')
    Ro = np.ones(order) * params.get('Ro')
    Cp = np.ones(order) * params.get('Cp')
    v_n = np.ones(order) * params.get('v_n')
    slew_rate = np.ones(order) * params.get('slew_rate')
    v_out_max = np.ones(order) * params.get('output_swing')
    v_out_min = -v_out_max
    
    # simulate SNR:
    try:
        GmC_analog_frontend = cb.GmC(
            analog_frontend, Cint, Ro, Cp, v_n, v_out_max, v_out_min, slew_rate
        )
    
        snr_GmC, amp_GmC, _ = GmC_analog_frontend.simulateSNR(osr)
        max_snr_GmC = max(snr_GmC)

        if not np.isnan(max_snr_GmC):
            print(f"Achieved SNR: {max_snr_GmC} db")
            return max_snr_GmC
        else:
            return None
    
    except (LinAlgError, ValueError) as e:
        print(f"Error with Gm-C parameters {params}: {e}")
        return None

In [6]:
# simulation test:

params = {'form': 'FB', 'order': 2, 'Bw': 20e3, 'osr': 64, 'nlev': 2.0, 'Hinf': 1.5, 'implementation': 'Active-RC', 'Cint': 1e-12, 'gm': 0.01, 'Ro': 1e4, 'Co': 1e-12}
snr = simulate_circuit('FB', 2, 'Active-RC', params)

C:\Users\frang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\signal\_ltisys.py:599: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  self.num, self.den = normalize(*system)
C:\Users\frang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\signal\_filter_design.py:1125: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  b, a = normalize(b, a)
INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Achieved SNR: 73.9091408220696 db


In [7]:
def filter_parameter_relationships(params, implementation):
    """
    Apply relationships-based filters to ensure that the parameters 
    comply with physical and stability constraints

    Args:
    - params (dict): The parameters set
    - implementation (str): The implementation type ('Active-RC', 'Gm-C')

    Returns:
    - cond (bool): True if the parameters are valid, False otherwise
    """
    
    # add relationships between parameters as needed:
    
    """
    Example:
    
    if implementation == 'Active-RC':
        gm = params.get('gm')
        Ro = params.get('Ro')

        if gm * Ro > 1e3:
            return False # invalid relationship

    elif implementation == 'Gm-C':
        gm = params.get('gm')
        Ro = params.get('Ro')

        if gm * Ro < 1e-2:
            return False # invalid relationship
    """
    
    return True # if no issues, return True to indicate the parameters are valid

In [8]:
# dataset generator test:

form = 'FB'
order = 2
implementation = 'Active-RC'
num_samples = 7
min_snr = 50

# define parameter ranges (set as function args better?):
if implementation == 'Active-RC':
    parameter_ranges = [
        (10e3, 10e9), # Bw (Bandwidth)
        [8, 16, 32, 64, 128, 256], # osr (Oversampling Ratio) - Discrete
        [2, 3, 4, 5], # nlev (Number of Levels) - Discrete
        (1.0, 2.0), # Hinf (Infinity Gain)
        (100e-15, 5e-12), # Cint (Integration Capacitance)
        (1e-6, 1e-3), # gm (Transconductance)
        (100e3, 10e6), # Ro (Output Resistance)
        (50e-15, 1e-12), # Co (Output Capacitance)
    ]
elif implementation == 'Gm-C':
    parameter_ranges = [
        (10e3, 10e9), # Bw (Bandwidth)
        [8, 16, 32, 64, 128, 256], # osr (Oversampling Ratio) - Discrete
        [2, 3, 4, 5], # nlev (Number of Levels) - Discrete
        (1.0, 2.0), # Hinf (Infinity Gain)
        (100e-15, 2e-12), # Cint (Integration Capacitance)
        (10e3, 1e6), # Ro (Output Resistance)
        (1e-15, 100e-15), # Cp (Capacitor)
        (10e-9, 1e-6), # v_n (Noise Voltage)
        (1e-6, 100e-6), # slew_rate (Slew Rate)
        (0.5, 3.0), # output_swing (Output Swing)
    ]

# define discrete parameters:
discrete_params = [
    (1, [8, 16, 32, 64, 128, 256]), # osr (discrete)
    (2, [2, 3, 4, 5]) # nlev (discrete)
]

dataset = generate_dataset(form, order, implementation, num_samples, min_snr, parameter_ranges, discrete_params)
print(dataset.head())

INFO:root:Simulating continuous-time analog frontend for sinusoidal input
C:\Users\frang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\signal\_ltisys.py:3137: ComplexWarning: Casting complex values to real discards the imaginary part
  yout[i, :] = (np.dot(system.C, xout[i, :]) +
C:\Users\frang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\signal\_ltisys.py:3141: ComplexWarning: Casting complex values to real discards the imaginary part
  yout[out_samples-1, :] = (np.dot(system.C, xout[out_samples-1, :]) +
INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Error with Active-RC parameters {'Bw': np.float64(7097446766.865976), 'osr': 64, 'nlev': 4, 'Hinf': np.float64(1.2901971117563211), 'Cint': np.float64(3.579581163631779e-12), 'gm': np.float64(0.0006109817879558087), 'Ro': np.float64(7509795.882788671), 'Co': np.float64(4.81969295730607e-13)}: The associated Hamiltonian pencil has eigenvalues too close to the imaginary axis


INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Error with Active-RC parameters {'Bw': np.float64(3524372896.915697), 'osr': 256, 'nlev': 2, 'Hinf': np.float64(1.544402154487626), 'Cint': np.float64(4.912388862891682e-12), 'gm': np.float64(7.658827816036257e-05), 'Ro': np.float64(3264340.8607567814), 'Co': np.float64(1.0887339215989456e-13)}: The associated Hamiltonian pencil has eigenvalues too close to the imaginary axis


INFO:root:Simulating continuous-time analog frontend for sinusoidal input


...

Error with Active-RC parameters {'Bw': np.float64(9642804456.7386), 'osr': 128, 'nlev': 3, 'Hinf': np.float64(1.8465767657469003), 'Cint': np.float64(3.939582191631325e-12), 'gm': np.float64(0.0005023556389005045), 'Ro': np.float64(5914781.266988719), 'Co': np.float64(3.179031369402818e-13)}: The associated Hamiltonian pencil has eigenvalues too close to the imaginary axis


INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Error with Active-RC parameters {'Bw': np.float64(7499982596.817185), 'osr': 16, 'nlev': 2, 'Hinf': np.float64(1.3012439215543468), 'Cint': np.float64(8.232861482062001e-13), 'gm': np.float64(0.000956892857010341), 'Ro': np.float64(3723791.2831714246), 'Co': np.float64(7.591402567656122e-13)}: Failed to find a finite solution.


INFO:root:Simulating continuous-time analog frontend for sinusoidal input


Achieved SNR: 4.716379049461771 db
Error with Active-RC parameters {'Bw': np.float64(4328138896.165132), 'osr': 32, 'nlev': 4, 'Hinf': np.float64(1.2241368211739845), 'Cint': np.float64(4.306152426764411e-12), 'gm': np.float64(0.00014557833468468166), 'Ro': np.float64(5538044.552408453), 'Co': np.float64(4.4256112999013353e-13)}: Failed to find a finite solution.

	Dataset saved as datasets/dataset_FB_2_Active-RC.csv

             Bw  osr  nlev      Hinf          Cint        gm            Ro  \
0  1.386141e+09  256     2  1.968385  2.029875e-12  0.000257  2.527794e+06   
1  1.644046e+09  128     4  1.145150  5.957712e-13  0.000900  8.639163e+06   
2  3.780387e+09  256     3  1.146855  1.914391e-12  0.000981  3.459372e+06   
3  5.951083e+09  256     5  1.936055  2.359900e-12  0.000845  8.982091e+06   
4  1.520502e+08   64     5  1.302211  1.533912e-13  0.000235  6.024396e+06   

             Co        SNR  
0  4.584401e-13  77.753803  
1  2.356851e-13  68.659580  
2  2.339022e-13  74.02